### With关键字

在Python中可以使用 `with` 关键字来使用 `context manager` 上下文管理器。

不正确地管理资源可能导致内存泄露，包括：
- 文件: 释放文件
- 锁: 释放锁
- 网络连接: 管理网络连接

```python
file = open("hello.txt", "w")
file.write("hello, world!")
file.close()
```

上面这段代码并不能保证文件被释放，如果在 `write()` 过程中抛出 exception，那么`close()` 就永远不会被执行。

有两种方式可以解决这个问题：
1. `try...finally` 代码块
2. `with` 代码块

### The `try...finally` approach
```python
# safely open the file
file = open('hello.txt', 'w')
try:
    file.write("hello, world!")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")
finally:
    # make sure to close the file after using it
    file.close()
```

the `finally` clause will guarantee that file is properly closed, even if an exception occurs during the call to `.write()` in the try clause.



### The `with` Statement Approach

> The Python `with` statement creates a **runtime context** that allows you to run a group of statements under the control of a context manager.

许多标准库支持 `with` 语句，比如 `open()` 允许你对文件对象使用 `with`

```python
with expression as target_var:
    do_something(target_var)
```

`expression` 必须返回一个对象，这个对象必须实现 context management protocol，也就是必须实现两个方法：
1. `__enter__()` 当with语句进入runtime context时执行
2. `__exit__()` 当离开with代码块时执行

Here's how the with statement processds when python runs into it

1. Call expression to obtain a context manager
2. Store the context manager's `__enter__()` and `__exit__()` methods for later use
3. Call `.__enter()__` on the context manager and bind its return value to `target_var` if provided
4. Execute the `with` code block
5. Call `.__exit__()` on the context manager when the with code block finishes

In this case, `.__enter__()`, typically provides the setup code. When the block ends, `.__exit__()` automatically gets called and closes the file for you, even if an exception is raised inside the with block.

在Python 3.1及以后，with可以创建多个 context manager

```python
with A() as a, B() as b:
    pass
```

比如你可以同时打开两个文件
```python
with open("input.txt") as in_file, open("output.txt", "w") as out_file:
    pass
```


### Perform High-Precision Calculation

Unlike built-int floating-point numbers, the `decimal` module provides a way to adjust the precision to use in a given calculation that involves `Decimal` numbers. The precision defautls to 28 places, but you can change it to meet your problem requirements.


In [6]:
from decimal import Decimal, localcontext

with localcontext() as ctx:
    ctx.prec = 42
    print(Decimal("1") / Decimal("42"))

# 重新变为28位小数
print(Decimal("1") / Decimal("42"))

0.0238095238095238095238095238095238095238095
0.02380952380952380952380952381


Here, `localcontext()` provides a context manager that creates a local decimal context and allows you to perform calculations using a custom precision. In the with code block ,you need to set `.prec` to the new precision you want to use, which is 42 places in the example above. When the `with` code block finishes, the precision is reset back to its default value, 28 places

### Handling Locks in Multithreaded Programs

另一个使用了 `with` 语句的标准库是 `threading.Lock`。 这个Lock类提供了源语锁，用来防止多线程下的race condition。

你可以用with语句来自动释放锁🔐。

```python
import threading

balance_lock = threading.Lock()
balance_lock.acquire()
try:
    # update the account balance
finally:
    balance_lock.release()

    
# use the with pattern
with balance_lock:
    # update the account balance here
```

### Testing for Exceptions with pytest

如果你使用 `pytest` 来测试你的代码，测试函数或者代码块在某些情况下抛出异常，可以使用 `pytest.raises()` 来提供一个context manager

```python
import pytest

with pytest.raises(ZeroDivisionError):
    1 / 0
    
favorites = {"fruit": "apple", "pet": "dog"}
with pytest.raises(KeyError):
    favorites["car"]
```
